In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv
%pip install neo4j
%pip install json-repair
%pip install py2neo
%pip install tree_sitter
%pip install --upgrade langchain
%pip install llama-index --upgrade
%pip show llama-index
%pip install yfiles-jupyter-graphs
%pip install --upgrade yfiles-jupyter-graphs
%pip install --upgrade jupyterlab ipywidgets
%pip install jupyterlab
%pip install sqlfluff

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Name: llama-index
Version: 0.11.14
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-ques

In [33]:
%pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import re

from dotenv import load_dotenv

load_dotenv()

/Users/larissakemicloud.com/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [3]:
os.environ["NEO4J_URI"] 
os.environ["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"]

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [94]:
from langchain.chains import GraphCypherQAChain, LLMChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Definir o template para geração de Cypher
CYPHER_GENERATION_TEMPLATE = """Objetivo: Gerar uma instrução Cypher para consultar o banco de dados Neo4J que possui uma estrutura de grafos representando um Schema de Banco de dados SQL.
Instruções:
- Verifique o esquema do banco de dados SQL consultando o Neo4j para obter os nomes das tabelas (nós) e colunas (propriedades).
- Converta qualquer termo no plural para o singular antes de gerar a query.
- Construa uma query Cypher para obter todas as Tabelas e Colunas que relacionem com o Prompt do usuário.
- Certifique-se de verificar os relacionamentos (chaves estrangeiras) no banco de dados gráfico.
- Não inclua explicações ou desculpas em suas respostas. Apenas forneça a query SQL gerada.
- Não responda a perguntas que solicitem qualquer coisa além da construção de uma instrução Cypher.
- Não inclua nenhum texto além da instrução Cypher gerada.
----------
Task: Com base no prompt de usuário abaixo faça os seguintes passos:
1. Extraia metadados/termos do prompt do usuário que sejam importantes ou contenham possíveis nomes de entidades e colunas.
2. Busque na Relação de tabelas (nós) existentes no banco, tabelas que possam satisfazer a query do usuário e utilize elas para formatar os termos utilizados na busca Cypher.
3. Gere uma query Cypher que consulte todas as tabelas ou colunas com esses nomes.
4. Considere consultas relativas (LIKE) em cima dos termos buscados.
5. Gere uma cópia da resposta da query gerada para que possa ser usada em um passo posterior, como a criação de uma query SQL.
----------
Exemplo de Query Cypher para ser utilizado:
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'usuario' OR c.id CONTAINS 'usuario' OR c.id CONTAINS 'ativo' OR c.id CONTAINS 'sistema'
RETURN t.id, c.id
----------
Relação do nome das tabelas existentes dentro do banco de dados Neo4j:
{tables}
----------
Schema banco de dados:
{schema}
----------
User prompt:
{query}
"""

# Criar o template para o prompt de Cypher
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "query", "tables"], template=CYPHER_GENERATION_TEMPLATE
)

# Conectar ao modelo da OpenAI
llm = ChatOpenAI(temperature=0)

# Obter o esquema do banco de dados Neo4j automaticamente
schema_data = graph.schema 

# Obter os rótulos (labels) das "tabelas" no banco de dados Neo4j usando a consulta Cypher correta
query_get_labels = "MATCH (t:Table) RETURN t.id"
#MATCH (t:Table)-[:HAS_COLUMN]->(n:Column) RETURN n.id, t.id

# Executar a consulta para obter as tabelas/rótulos usando graph.query
result_labels = graph.query(query_get_labels)

# Extrair apenas os nomes das tabelas (rótulos) do resultado Cypher
table_names = [result['t.id'] for result in result_labels]

# Juntar os nomes das tabelas em um formato que o modelo entenda
tables_list_str = ", \n".join(table_names)
print(f"Tabelas do Neo4j: {tables_list_str}")

# Criar o chain de perguntas ao Neo4j
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,  # Passa a conexão ao banco de dados Neo4j
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    prompt=SQL_TEMPLATE,
    allow_dangerous_requests=True,  # Permitir consultas perigosas
)

# Definir a query
query = "Quais sao os usuarios ativos do sistema?"

# Executar a cadeia para gerar a consulta Cypher a partir do schema e da query
cypher_result = cypher_chain.run({
    "schema": schema_data,
    "tables": tables_list_str,
    "query": query,
})

# Use o Full Context como a resposta final (ou passe para o próximo passo)
print(f"Full Context: {cypher_result}")

Tabelas do Neo4j: Controledeacessoapitokens, 
Controledeacessousuario, 
Empresa, 
Controledeacessogrupo, 
Controledeacessogrupousuario, 
Controledeacessomenu, 
Controledeacessomodulotela, 
Controledeacessomodulo, 
Controledeacessotela, 
Controledeacessopermissao, 
Controledeacessorecurso, 
Controledeacessopermissaomenu, 
Usuario, 
Controledeacessousuariocliente


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'Usuario' OR c.id CONTAINS 'Usuario' OR c.id CONTAINS 'ativo' OR c.id CONTAINS 'sistema'
RETURN t.id, c.id
Full Context:
[{'t.id': 'Controledeacessoapitokens', 'c.id': 'Id_Usuario'}, {'t.id': 'Controledeacessomenu', 'c.id': 'Aplicativo'}, {'t.id': 'Controledeacessorecurso', 'c.id': 'Aplicativo'}, {'t.id': 'Controledeacessorecurso', 'c.id': 'Id_Usuario'}, {'t.id': 'Controledeacessopermissaomenu', 'c.id': 'Id_Usuario'}]

> Finished chain.
Full Context: Eu não sei a resposta.


In [74]:
# Definir o template para geração de SQL
SQL_TEMPLATE = """Objetivo: Responda sempre com uma query SQL que reflita a pergunta solicitada, sem usar o caractere '*' na construção das queries.

Contexto:
- `t.id` representa o nome da tabela. Por exemplo, se `t.id` for 'Controledeacessousuario', isso significa que o nome da tabela é 'Controledeacessousuario'.
- `c.id` representa o nome da coluna. Por exemplo, se `c.id` for 'Status', isso significa que a coluna que você deve usar na query SQL é 'Status'.

Instruções:
- Use somente o nome da tabela e o nome da coluna fornecidos no `Full Context` abaixo para construir a query SQL.
- Não use `t.id` ou `c.id` diretamente na query. Apenas extraia o nome da tabela (de `t.id`) e o nome da coluna (de `c.id`).
- Não permita retornos ou queries de 'SELECT *' de uma tabela.

Full Context: {result}
----------
User prompt:
{query}
"""

# Criar o template para o prompt de SQL
SQL_GENERATION_PROMPT = PromptTemplate(
    input_variables=["result", "query"], template=SQL_TEMPLATE
)

# Agora, use o Full Context para gerar uma query SQL correspondente
sql_chain = LLMChain(
    llm=llm,
    prompt=SQL_GENERATION_PROMPT,
    verbose=True
)

# Executar a cadeia para gerar a query SQL
sql_result = sql_chain.run({
    "result": cypher_result, 
    "query": query
})

# Imprimir a consulta SQL gerada
print(f"SQL gerada: {sql_result}")



> Entering new LLMChain chain...
Prompt after formatting:
Objetivo: Responda sempre com uma query SQL que reflita a pergunta solicitada, sem usar o caractere '*' na construção das queries.

Contexto:
- `t.id` representa o nome da tabela. Por exemplo, se `t.id` for 'Controledeacessousuario', isso significa que o nome da tabela é 'Controledeacessousuario'.
- `c.id` representa o nome da coluna. Por exemplo, se `c.id` for 'Status', isso significa que a coluna que você deve usar na query SQL é 'Status'.

Instruções:
- Use somente o nome da tabela e o nome da coluna fornecidos no `Full Context` abaixo para construir a query SQL.
- Não use `t.id` ou `c.id` diretamente na query. Apenas extraia o nome da tabela (de `t.id`) e o nome da coluna (de `c.id`).
- Não permita retornos ou queries de 'SELECT *' de uma tabela.

Full Context: Eu não sei a resposta.
----------
User prompt:
Quais sao os usuarios ativos do sistema?


> Finished chain.
SQL gerada: SELECT Usuario
FROM Sistema
WHERE Status = '